In [1]:
import time
from datetime import datetime

from fmi_mlc import fmi_gym
from mqtt_publisher import MQTTPublisher

# import the config file generated in export process this contains the parameters for startung the simulation so user does not need to write them
from config import parameter as dfmu

In [2]:
pub=MQTTPublisher(host="localhost", port=1883)

limits = [
    {"min": float(lo), "max": float(hi), "step": 0.1}
    for lo, hi in zip(dfmu['action_min'], dfmu['action_max'])
]
#starts the environment with the parameters on the config file and starts the warmup period
env = fmi_gym(dfmu)
env.reset()

array([24.21019234, 23.6501538 , 23.67073513, 24.19226204, 22.96352637,
       22.73993846, 23.05817025])

In [3]:
#main loop of the simulation

default_action = {'Cool_SP_Ingresso_fmu': 26.0, 'Cool_SP_Camera1_fmu': 27.0, 'Cool_SP_Camera2_fmu': 28.0, 'Cool_SP_Camera3_fmu': 29.0, 'Cool_SP_Bagno1_fmu': 30.0, 'Cool_SP_Bagno2_fmu': 26.5, 'Cool_SP_Cucina_fmu': 29.5}

#we create a different defult action for each actuator to be applied in order to see temperature behaviour 

while env.fmu_time < dfmu['fmu_final_time']:

    #here you could implement your own algorithm with your own actions and conditionals so you can create custom control strategies for your model
    #this example was only for testing data exchange between python and energyplus via the FMU
    
    obs, *_ = env.step(default_action) #increments a step each timestep in the idf for this example timestep is a value of 6
    data = {n: float(v) for n, v in zip(dfmu['observation_names'], obs)}
    
    pub.publish_observations(data, timestamp=env.fmu_time) #publish observation data to MQTT

    pub.publish_actions(default_action, timestamp=env.fmu_time) #publish actions data to MQTT

    #time.stop(1) if you want to stop the simulation for a given time add this line here

# close the conection to influx
pub.close()

print("¡Simulation completed")

¡Simulation completed
